In [161]:
import xml.etree.ElementTree as ET
import re
from os import listdir
topic_files = listdir('./topics/')
for file in topic_files:
    file = './topics/' + file
# The regex used to capture the words segements
p_pattern = r'#id\((\w+\.\w\.\w+)\)\.{2}id\((\w+\.\w\.\w+)\)'
s_pattern = r'#id\((\w+\.\w\.\w+)\)'
two_pattern = r'#id\((\w+\.\w\.\w+)(\d+)\)\.{2}id\((\w+\.\w\.\w+?)(\d+)\)'
one_pattern = r'#id\((\w+\.\w\.\w+)(\d+)\)'
name_space = {'nite':'http://nite.sourceforge.net/'}
words_pattern = '(.+?)words'

In [280]:
def parse_transcript(topic):
    topic_tree = ET.parse(topic)
    topic_root = topic_tree.getroot()
    topic_name = topic_root.attrib['{http://nite.sourceforge.net/}id']
    word_segments = []

    #get word segments without the paragraph bounds
    for topic in topic_root.iter('topic'):
        for child in topic.findall('nite:child',name_space):
            try:
                word_segments.append(list(re.search(p_pattern,child.attrib['href']).groups()))
            except:
                word_segments.append(list(re.search(s_pattern,child.attrib['href']).groups()))
    #get topic bounds
    bounds = set()
    for topic in topic_root.findall('topic'):
        try:
            matched = re.search(p_pattern,topic.findall('.//nite:child[last()]',name_space)[-1].attrib['href'])
            bounds.add(matched.groups())
        except:
            matched = re.search(s_pattern,topic.findall('.//nite:child[last()]',name_space)[-1].attrib['href'])
            bounds.add(matched.groups())        
    #add topical bounds
    for seg in word_segments:
        if tuple(seg) in bounds:
            seg.append("**********")
    #get paragraph bounds
    seg_file_pattern1 = r'\w{7}\.\w\.'
    seg_file_pattern2 = r'\w{6}\.\w\.'
    wseg_with_parapgraph = []
    for seg in word_segments:
        if len(seg) == 1:
            wseg_with_parapgraph.append(seg)
        else:# the length of segment is 2 or 3, 3 means its a topical boundary
            try:
                file_name = re.match(seg_file_pattern1, seg[0]).group(0) + "segments.xml"
            except:
                file_name = re.match(seg_file_pattern2, seg[0]).group(0) + "segments.xml"
            tree =  ET.parse('./segments/'+file_name)
            root = tree.getroot()
            paras = []
            start = False
            for segment in root.findall('.//nite:child', name_space):
                if start:
                    try:
                        paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                    except:
                        paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
                if seg[0] in segment.attrib['href']:
                    start = True
                    try:
                        paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                    except:
                        paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
                if seg[1] in segment.attrib['href']:
                    break
            if len(paras) == 0:
                paras.append(seg[0] + ',' + seg[1])
            if seg[-1] == '**********':
                paras.append('**********')
            wseg_with_parapgraph.append(paras)
    #start of writing txt process        
    with open('./txt_files/'+ topic_name+'.txt','w') as f:
        for e in wseg_with_parapgraph:
            for s in e:
                if s == '**********':
                    f.write(s + '\n')
                else:   
                    word_file = './words/'+ re.match(words_pattern,s).group() + '.xml'
                    tree = ET.parse(word_file)
                    root = tree.getroot()
                    sentence = s.split(',')
                    if len(sentence) == 1:# even single word sentence will be length of 2 here, 1 means vocal which can be dropped
                        pass
                    else:# length is 2, have start and end word
                        start = sentence[0]
                        end = sentence[1]
                        no_word = 0
                        begin = False
                        for word in root.findall('*'):
                            if begin:
                                if word.text:
                                    no_word += f.write(word.text + ' ') 
                                else:
                                    pass
                            if word.attrib['{http://nite.sourceforge.net/}id'] == start:
                                begin = True
                                if word.text:
                                    no_word = f.write(word.text + ' ') 
                                else:
                                    pass
                            if word.attrib['{http://nite.sourceforge.net/}id'] == end:      
                                if no_word:
                                    f.write('\n')
                                break

In [281]:
for file in topic_files:
    file = './topics/' + file
    parse_transcript(file)

In [271]:
if 0:
    print('y')
else:
    print('n')

n


### !!! Runing time is excessively long, better reduce iteration to optimize the running efficiency

In [279]:
#test the parsing process on one transcript
parse_transcript('./topics/ES2002a.topic.xml')

In [246]:
if None:
    pass
else:
    print('a')

a


In [177]:
'ES2002a.B.words0,ES2002a.B.words1'.split(',')

['ES2002a.B.words0', 'ES2002a.B.words1']

re.match(words_pattern,'ES2002a.B.words0,ES2002a.B.words1').group()

In [211]:
tree = ET.parse('./words/ES2002a.B.words.xml')
root = tree.getroot()
for word in root.findall('*'):
    if word.attrib['{http://nite.sourceforge.net/}id'] == 'ES2002a.B.words4':
        print(word.text)

None


['ES2002a.B.words275']

ES2002a.B.words0,ES2002a.B.words1

In [82]:
bounds

{('ES2002d.B.words0', 'ES2002d.B.words170'),
 ('ES2002d.B.words1612', 'ES2002d.B.words1613'),
 ('ES2002d.B.words2586', 'ES2002d.B.words2589'),
 ('ES2002d.B.words2873', 'ES2002d.B.words2885'),
 ('ES2002d.B.words446', 'ES2002d.B.words473')}

In [83]:
for seg in word_segments:
    if tuple(seg) in bounds:
        seg.append("**********")

In [200]:
def a(topic):
    topic_tree = ET.parse(topic)
    topic_root = topic_tree.getroot()
    topic_name = topic_root.attrib['{http://nite.sourceforge.net/}id']
    word_segments = []

    #get word segments without the paragraph bounds
    for topic in topic_root.iter('topic'):
        for child in topic.findall('nite:child',name_space):
            try:
                word_segments.append(list(re.search(p_pattern,child.attrib['href']).groups()))
            except:
                word_segments.append(list(re.search(s_pattern,child.attrib['href']).groups()))
    #get topic bounds
    bounds = set()
    for topic in topic_root.findall('topic'):
        try:
            matched = re.search(p_pattern,topic.findall('.//nite:child[last()]',name_space)[-1].attrib['href'])
            bounds.add(matched.groups())
        except:
            matched = re.search(s_pattern,topic.findall('.//nite:child[last()]',name_space)[-1].attrib['href'])
            bounds.add(matched.groups())        
    #add topical bounds
    for seg in word_segments:
        if tuple(seg) in bounds:
            seg.append("**********")
    #get paragraph bounds
    seg_file_pattern1 = r'\w{7}\.\w\.'
    seg_file_pattern2 = r'\w{6}\.\w\.'
    wseg_with_parapgraph = []
    for seg in word_segments:
        if len(seg) == 1:
            wseg_with_parapgraph.append(seg)
        else:# the length of segment is 2 or 3, 3 means its a topical boundary
            try:
                file_name = re.match(seg_file_pattern1, seg[0]).group(0) + "segments.xml"
            except:
                file_name = re.match(seg_file_pattern2, seg[0]).group(0) + "segments.xml"
            tree =  ET.parse('./segments/'+file_name)
            root = tree.getroot()
            paras = []
            start = False
            for segment in root.findall('.//nite:child', name_space):
                if start:
                    try:
                        paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                    except:
                        paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
                if seg[0] in segment.attrib['href']:
                    start = True
                    try:
                        paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                    except:
                        paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
                if seg[1] in segment.attrib['href']:
                    break
            if len(paras) == 0:
                paras.append(seg[0] + ',' + seg[1])
            if seg[-1] == '**********':
                paras.append('**********')
            wseg_with_parapgraph.append(paras)
    return wseg_with_parapgraph

In [213]:
a('./topics/ES2002a.topic.xml')

[['ES2002a.B.words0,ES2002a.B.words1',
  'ES2002a.B.words2,ES2002a.B.words3',
  'ES2002a.B.words4,ES2002a.B.words71'],
 ['ES2002a.D.words0,ES2002a.D.words1', 'ES2002a.D.words2,ES2002a.D.words3'],
 ['ES2002a.A.words0,ES2002a.A.words12'],
 ['ES2002a.B.words72,ES2002a.B.words73'],
 ['ES2002a.D.words4,ES2002a.D.words11'],
 ['ES2002a.C.words0,ES2002a.C.words7'],
 ['ES2002a.D.words12,ES2002a.D.words13', '**********'],
 ['ES2002a.B.words74,ES2002a.B.words109',
  'ES2002a.B.words110,ES2002a.B.words116',
  'ES2002a.B.words117,ES2002a.B.words192'],
 ['ES2002a.A.words13,ES2002a.A.words48'],
 ['ES2002a.B.words193,ES2002a.B.words194'],
 ['ES2002a.D.words14,ES2002a.D.words15',
  'ES2002a.D.words16,ES2002a.D.words23'],
 ['ES2002a.C.words8,ES2002a.C.words9'],
 ['ES2002a.D.words24,ES2002a.D.words25'],
 ['ES2002a.B.words195,ES2002a.B.words268',
  'ES2002a.B.words269,ES2002a.B.words272',
  'ES2002a.B.words273,ES2002a.B.words274',
  'ES2002a.B.words275',
  'ES2002a.B.words276,ES2002a.B.words279',
  'ES200

In [270]:
with open('./test.txt','w') as f:
    a = f.write('1')
    a += f.write('2')
    a += f.write('3')
    print(a)

3


In [84]:
#now we know where the topical boundaries are
word_segments

[['ES2002d.B.words0', 'ES2002d.B.words170', '**********'],
 ['ES2002d.A.words0', 'ES2002d.A.words1'],
 ['ES2002d.D.words0', 'ES2002d.D.words86'],
 ['ES2002d.A.words2', 'ES2002d.A.words9'],
 ['ES2002d.B.words171', 'ES2002d.B.words176'],
 ['ES2002d.D.words87', 'ES2002d.D.words107'],
 ['ES2002d.C.words0', 'ES2002d.C.words2'],
 ['ES2002d.A.words10', 'ES2002d.A.words46'],
 ['ES2002d.C.words3', 'ES2002d.C.words6'],
 ['ES2002d.A.words47', 'ES2002d.A.words59'],
 ['ES2002d.C.words7', 'ES2002d.C.words12'],
 ['ES2002d.B.words177', 'ES2002d.B.words187'],
 ['ES2002d.D.words108'],
 ['ES2002d.A.words60', 'ES2002d.A.words132'],
 ['ES2002d.C.words13', 'ES2002d.C.words18'],
 ['ES2002d.B.words188', 'ES2002d.B.words192'],
 ['ES2002d.C.words19', 'ES2002d.C.words22'],
 ['ES2002d.A.words133', 'ES2002d.A.words164'],
 ['ES2002d.C.words23', 'ES2002d.C.words32'],
 ['ES2002d.B.words193', 'ES2002d.B.words198'],
 ['ES2002d.A.words165', 'ES2002d.A.words169'],
 ['ES2002d.C.words33', 'ES2002d.C.words43'],
 ['ES2002d.B

### ---- A test of how to capture paragraph boundaries

In [85]:
tree = ET.parse('./segments/ES2002b.B.segments.xml')
root = tree.getroot()

In [86]:
para = []
start = False
for segment in root.findall('.//nite:child', name_space):
    if start:
        para.append(re.search(p_pattern, segment.attrib['href']).groups())
    if 'ES2002b.B.words0' in segment.attrib['href']:
        para.append(re.search(p_pattern, segment.attrib['href']).groups())
        start = True
    if 'ES2002b.B.words128' in segment.attrib['href']:
        break

In [87]:
para

[('ES2002b.B.words0', 'ES2002b.B.words4'),
 ('ES2002b.B.words5', 'ES2002b.B.words7'),
 ('ES2002b.B.words8', 'ES2002b.B.words9'),
 ('ES2002b.B.words10', 'ES2002b.B.words128')]

In [88]:
'ES2002b.B.words0' in "ES2002b.B.words.xml#id(ES2002b.B.words0)..id(ES2002b.B.words4)"

True

### Then we need to figure out paragraph bounds

### First we try to get the complete transcript for one topic xml file -- "`ES2002b.topic.xml`"

In [89]:
seg_file_pattern = r'\w{7}\.\w\.'

### Some more complicated situation that we need to be able to  address:
**1. The start word of a segment from the topic file(`length 2`) is in the interval of segment in segments files, the end word matches the one in the segments file**

**2. Start word matches but the end word falls in the interval**
3. Both start word and end word are in the interval
4. The segment in segments is single length and match only the start word of a segment in topic file
5. The segment in segments is single length and match only the end word of a segment in topic file

### It's much easier to deal with the stareting and ending intervals in only numeric mode

                try:
                    paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                except:
                    paras.append(re.search(s_pattern, segment.attrib['href']).group(1))

In [116]:
wseg_with_parapgraph1 = []
for seg in word_segments:
    if len(seg) == 1:
        wseg_with_parapgraph1.append(seg)
    else:# the length of segment is 2 or 3, 3 means its a topical boundary
        file_name = re.match(seg_file_pattern, seg[0]).group(0) + "segments.xml"
        tree =  ET.parse('./segments/'+file_name)
        root = tree.getroot()
        paras = []
        start = False
        for segment in root.findall('.//nite:child', name_space):
            if start:
                try:
                    paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                except:
                    paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
            if seg[0] in segment.attrib['href']:
                start = True
                try:
                    paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                except:
                    paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
            if seg[1] in segment.attrib['href']:
                break
        if len(paras) == 0:
            paras.append(seg[0] + ',' + seg[1])
        if seg[-1] == '**********':
            paras.append('**********')
        wseg_with_parapgraph1.append(paras)        

['ES2002d.B.words474', 'ES2002d.B.words502']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words490)..id(ES2002d.B.words502)'}
['ES2002d.B.words742', 'ES2002d.B.words767']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words720)..id(ES2002d.B.words767)'}
['ES2002d.B.words1614', 'ES2002d.B.words1632']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words1612)..id(ES2002d.B.words1632)'}
['ES2002d.B.words1695', 'ES2002d.B.words1710']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words1685)..id(ES2002d.B.words1710)'}
['ES2002d.B.words1913', 'ES2002d.B.words1917']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words1910)..id(ES2002d.B.words1917)'}
['ES2002d.B.words2088', 'ES2002d.B.words2127']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words2076)..id(ES2002d.B.words2127)'}
['ES2002d.B.words2590', 'ES2002d.B.words2645']
{'href': 'ES2002d.B.words.xml#id(ES2002d.B.words2586)..id(ES2002d.B.words2645)'}


# REVISED VERSION TO DEAL WITH SEGMENTS SPLIT IN TWO SUBTOPICS
wseg_with_parapgraph = []
for seg in word_segments:
    if len(seg) == 1:
        wseg_with_parapgraph.append(seg)
    else:# the length of segment is 2 or 3, 3 means its a topical boundary
        file_name = re.match(seg_file_pattern, seg[0]).group(0) + "segments.xml"
        tree =  ET.parse('./segments/'+file_name)
        root = tree.getroot()
        paras = []
        #start = False
        for segment in root.findall('.//nite:child', name_space):
            #if start:
                #try:
                    #paras.append(','.join(re.search(p_pattern, segment.attrib['href']).groups()))
                #except:
                    #paras.append(re.search(s_pattern, segment.attrib['href']).group(1))
            if re.search(p_pattern, segment.attrib['href']):#this means the segment has length of 2
                if seg[0] >= re.search(p_pattern, segment.attrib['href']).group(1) and seg[1] >= re.search(p_pattern, segment.attrib['href']).group(2):
                    paras.append(seg[0] + ',' + re.search(p_pattern, segment.attrib['href']).group(2))                   
                if seg[0] >= re.search(p_pattern, segment.attrib['href']).group(1) and seg[1] < re.search(p_pattern, segment.attrib['href']).group(2):
                    paras.append(seg[0] + ',' + seg[1])  
                if seg[0] < re.search(p_pattern, segment.attrib['href']).group(1) and seg[1] >= re.search(p_pattern, segment.attrib['href']).group(2):
        if seg[-1] == '**********':
            paras.append('**********')
        if len(paras) == 0:
            print(seg)
            print(segment.attrib)
        wseg_with_parapgraph.append(paras)

                start = True
                try:
                    paras.append(re.search(p_pattern, segment.attrib['href']).group(1) + ',' + seg[0])
                except:
                    paras.append(seg[0])
            try:    
                if seg[1] <= re.search(p_pattern, segment.attrib['href']).group(2):
                    break
            except:
                if seg[1] <= re.search(s_pattern, segment.attrib['href']).group(1):
                    break

In [91]:
re.match('\w+','abc').group(0)

'abc'

In [92]:
wseg_with_parapgraph1

[['ES2002d.B.words0,ES2002d.B.words80',
  'ES2002d.B.words81,ES2002d.B.words170',
  '**********'],
 ['ES2002d.A.words0,ES2002d.A.words1'],
 ['ES2002d.D.words0,ES2002d.D.words86'],
 ['ES2002d.A.words2,ES2002d.A.words9'],
 ['ES2002d.B.words171,ES2002d.B.words172',
  'ES2002d.B.words173,ES2002d.B.words174',
  'ES2002d.B.words175,ES2002d.B.words176'],
 ['ES2002d.D.words87,ES2002d.D.words107'],
 ['ES2002d.C.words0,ES2002d.C.words2'],
 ['ES2002d.A.words10,ES2002d.A.words46'],
 ['ES2002d.C.words3,ES2002d.C.words6'],
 ['ES2002d.A.words47,ES2002d.A.words59'],
 ['ES2002d.C.words7,ES2002d.C.words12'],
 ['ES2002d.B.words177,ES2002d.B.words187'],
 ['ES2002d.D.words108'],
 ['ES2002d.A.words60,ES2002d.A.words132'],
 ['ES2002d.C.words13,ES2002d.C.words14',
  'ES2002d.C.words15,ES2002d.C.words16',
  'ES2002d.C.words17,ES2002d.C.words18'],
 ['ES2002d.B.words188,ES2002d.B.words192'],
 ['ES2002d.C.words19,ES2002d.C.words22'],
 ['ES2002d.A.words133,ES2002d.A.words164'],
 ['ES2002d.C.words23,ES2002d.C.words

In [111]:
with open('./txt_files/'+ topic_name+'.txt','w') as f:
    for e in wseg_with_parapgraph1:
        for s in e:
            f.write(s + '\n')

### ------- a break line for automating all files----

In [5]:
from os import listdir
topic_files = listdir('./topics/')
seg_files = listdir('./segments/')
words_files = listdir('./words/')

In [283]:
len(listdir('./txt_files/'))

140

In [119]:
with open('./test.txt','w') as f:
    for e in [[],'1','2']:
         for s in e:
            f.write(s+'\n')